In [81]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://opendata.arcgis.com/datasets/5b5c745e0f1f48e7a53acec63a0022ab_0.csv')


In [84]:
# Working columns

df_new = df[['SEVERITYCODE', 'WEATHER', "LIGHTCOND", 'ROADCOND']]
from sklearn.preprocessing import LabelEncoder
df_new.dropna(inplace = True)


df_new['SEVERITYCODE'] = LabelEncoder().fit_transform(df_new['SEVERITYCODE'])
df_new["LIGHTCOND_CAT"] = LabelEncoder().fit_transform(df_new['LIGHTCOND'])
df_new["WEATHER_CAT"] = LabelEncoder().fit_transform(df_new['WEATHER'])
df_new["ROADCOND_CAT"] = LabelEncoder().fit_transform(df_new['ROADCOND'])

# Define X , y

X = df_new[['WEATHER_CAT', 'LIGHTCOND_CAT', 'ROADCOND_CAT']].values
y = df_new['SEVERITYCODE'].values

# Standartization

from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

# Train-test 

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 191)
X_train.shape




/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda

(155759, 3)

In [85]:
# Desicion Tree Model
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)



from sklearn.metrics import classification_report, jaccard_similarity_score, f1_score, log_loss
print(jaccard_similarity_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))
print(classification_report(y_test, y_pred))


0.6844889573703133
0.20338508258188467
              precision    recall  f1-score   support

           1       0.68      1.00      0.81     26659
           2       0.31      0.00      0.00     11609
           3       0.00      0.00      0.00       596
           4       0.00      0.00      0.00        76

   micro avg       0.68      0.68      0.68     38940
   macro avg       0.25      0.25      0.20     38940
weighted avg       0.56      0.68      0.56     38940



/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [80]:
# Logistic regressiion
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=6, solver = 'liblinear').fit(X_train, y_train)
LR_pred = LR.predict(X_test)
LR_proba = LR.predict_proba(X_test)


print(jaccard_similarity_score(y_test, LR_pred))
print(f1_score(y_test, LR_pred, average = 'macro'))
print(log_loss(y_test, LR_proba))

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6846173600410889
0.20319669507157123


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


ValueError: y_true and y_pred contain different number of classes 4, 5. Please provide the true labels explicitly through the labels argument. Classes found in y_true: ['1' '2' '2b' '3']